# Day 8 - Lab 1: Vision-Enabled UI/UX Agents (Solution)

**Objective:** Use multi-modal vision models to generate a frontend UI from a design image, and then use a second agent to perform an automated design review.

**Introduction:**
This solution notebook provides the complete prompts and expected outputs for the vision-based frontend development lab. It demonstrates a full workflow from image-to-code, refactoring, and AI-powered design review.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

**Explanation:**
This setup block is crucial. It ensures our helper utilities are importable and initializes the client for a specific vision-capable model. For this lab, a powerful multi-modal model like `gpt-4o` is required to accurately interpret the design image.

In [2]:
import sys
import os

# Add the project's root directory to the Python path
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_vision_completion, get_completion, save_artifact, clean_llm_output
from IPython.display import Image, display, Code, Markdown

# Ensure you select a vision-capable model
client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")

if not model_name:
    print("Could not set up a valid LLM client. Please check your .env file and utils.py configuration.")

✅ LLM Client configured: Using 'openai' with model 'gpt-4o'


## Step 2: The Design Screenshot

**Explanation:**
We define the URL of the target design. This image will be the primary context for our vision-powered agents.

In [5]:
login_form_url = "https://i.imgur.com/s42SYz6.png"
display(Image(url=login_form_url))

## Step 3: The Challenges - Solutions

### Challenge 1 (Foundational): Generating a Monolithic UI Component

**Explanation:**
This prompt is a direct instruction to the vision model. We assign it the `persona` of an expert frontend developer. We provide the `context` (the image) and the `format` requirements (single React component, Tailwind CSS). This guides the model to produce a single, self-contained code block that replicates the design.

In [ ]:
generate_ui_prompt = f"""
You are an expert frontend developer specializing in React and Tailwind CSS.

Your task is to analyze the provided image of a login form and write the code for a single, self-contained React component that accurately replicates its design and layout.

**Requirements:**
- Use functional components.
- Use Tailwind CSS for all styling. Do not use custom CSS or style tags.
- Make sure the component is accessible, using appropriate HTML tags and attributes.
- The output should be only the raw JSX code for the component.
"""

print("--- Generating Monolithic UI Component ---")
generated_monolithic_code = ""
if model_name:
    generated_monolithic_code = get_vision_completion(generate_ui_prompt, login_form_url, client, model_name, api_provider)
    cleaned_code = clean_llm_output(generated_monolithic_code, language='jsx')
    display(Code(cleaned_code, language='jsx'))
    save_artifact(cleaned_code, "app/day8_login_monolithic.jsx")
else:
    print("Skipping UI generation because no valid model is configured.")

### Challenge 2 (Intermediate): Refactoring into Reusable Components

**Explanation:**
This prompt focuses on code quality. We provide the previously generated code as context and ask the LLM to act as a senior developer who values clean, component-based architecture. The instruction to create smaller, reusable components like `<StyledButton>` and `<InputWithIcon>` guides the LLM to perform a specific, best-practice refactoring task. Note that this is a text-only task, so we use the standard `get_completion` helper.

In [ ]:
refactor_ui_prompt = f"""
You are a senior frontend developer who champions clean, component-based architecture.

The following React code is a single, monolithic component. Your task is to refactor it.

**Refactoring Requirements:**
1.  Create smaller, reusable sub-components where appropriate. For example, the button and the input fields are good candidates for their own components (e.g., `InputWithIcon`, `LoginButton`).
2.  The final output should be a single file containing the definitions for the new, smaller components and the main `LoginForm` component that uses them.
3.  Ensure the final visual output remains identical to the original.

**Original Code:**
```jsx
{cleaned_code}
```

Output only the raw, complete, refactored JSX code.
"""

print("--- Refactoring UI into Components ---")
cleaned_refactored_code = ""
if 'cleaned_code' in locals() and cleaned_code:
    refactored_code = get_completion(refactor_ui_prompt, client, model_name, api_provider)
    cleaned_refactored_code = clean_llm_output(refactored_code, language='jsx')
    display(Code(cleaned_refactored_code, language='jsx'))
    save_artifact(cleaned_refactored_code, "app/day8_login_refactored.jsx")
else:
    print("Skipping refactoring because monolithic code was not generated.")

### Challenge 3 (Advanced): The AI UI/UX Critic Agent

**Explanation:**
This prompt demonstrates an AI-on-AI workflow for quality assurance. We give this new agent the persona of a meticulous UI/UX designer. Its task is to perform a comparative analysis. It receives two pieces of context: the original design `image` and the `code`. The prompt specifically asks it to compare the *likely rendered output* of the code to the design and list inconsistencies. This automates a tedious and subjective part of the development process.

In [ ]:
critic_agent_prompt = f"""
You are a meticulous UI/UX designer with a sharp eye for detail. Your task is to perform a design review.

You will be given two things:
1.  An image of the original design mockup.
2.  The React code that was generated to implement that design.

Your job is to compare the likely visual output of the code to the original design image. Identify and list any visual inconsistencies, no matter how small. Pay close attention to:
- Spacing and padding
- Font sizes and weights
- Color hex codes
- Border radius and shadows
- Overall layout and alignment

**Generated Code:**
```jsx
{cleaned_refactored_code}
```

Provide your feedback as a markdown-formatted list.
"""

print("--- Invoking UI/UX Critic Agent ---")
if cleaned_refactored_code:
    design_review = get_vision_completion(critic_agent_prompt, login_form_url, client, model_name, api_provider)
    display(Markdown(design_review))
    save_artifact(design_review, "artifacts/design_review.md")
else:
    print("Skipping critic agent because refactored code is not available.")

## Lab Conclusion

Fantastic! You have completed a full, end-to-end frontend development workflow using multiple AI agents. You used a vision-powered agent to generate code from a design, a refactoring agent to improve the code's structure, and a critic agent to perform an automated design review. This powerful combination of skills can dramatically accelerate the process of turning visual ideas into functional user interfaces.

> **Key Takeaway:** The workflow of **Generate -> Refactor -> Critique** is a powerful AI-assisted development pattern. Using specialized agents for each step allows you to rapidly create a first draft, improve its quality, and then automatically check it for correctness, significantly speeding up the iteration cycle.